In [10]:
import cv2 as cv
import numpy as np
import os
import tensorflow as tf
import keras
import inkml2img
import re
import json
import xml.etree.ElementTree as ET

In [2]:
# Convert .inkml files to .jpg images
PATH = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\TrainINKML"
DIR_PATH = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\images"
dataSets = os.listdir(PATH)
arr = [[] for _ in range(len(dataSets))]
print(dataSets)
for i in range(len(dataSets)):
    arr[i] = os.listdir(PATH+'\\'+dataSets[i])

'''for t in arr[i]:
    if ('.lg' not in t):
        inkml2img.inkml2img(PATH+'\\'+dataSets[i]+'\\'+t,DIR_PATH+'\\'+t[:-5]+'jpg')
        print(t)'''

['expressmatch', 'extension', 'HAMEX', 'KAIST', 'MathBrush', 'MfrDB']


"for t in arr[i]:\n    if ('.lg' not in t):\n        inkml2img.inkml2img(PATH+'\\'+dataSets[i]+'\\'+t,DIR_PATH+'\\'+t[:-5]+'jpg')\n        print(t)"

In [41]:
# get the latex expressions of each .inkml file
DIR_PATH = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\TrainINKML"
LABELS_PATH = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML"
# Dictionary where each .inkml file info will be store
items = {}
index = 0
count = 0

for t in arr:
    for i in t:
        if (".lg" not in i):
            imgPath = DIR_PATH+"\\"+ dataSets[index]+ "\\"+i
            tree = ET.parse(imgPath)
            root = tree.getroot()
            count =0
            for item in root:
                if item.text:
                    if "$" in item.text:
                        current = item.text
                        items[DIR_PATH+"\\"+ dataSets[index]+ "\\"+i] = current[1:-1]
                        break
                    if "\\" in item.text and dataSets[index] == "MathBrush":
                        current = item.text
                        items[DIR_PATH+"\\"+ dataSets[index]+ "\\"+i] = current[1:-1]
                        break
                    if (count ==1 and dataSets[index]=="KAIST" and len(item.text)>2):
                        current = item.text
                        items[DIR_PATH+"\\"+ dataSets[index]+ "\\"+i] = current
                        break
                count +=1
        else:
            print(i)
    index +=1 

for key in items:
    if ('l o g' in items[key]):
        items[key] = items[key].replace('l o g', '\\log')

with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\labels.json", 'w') as f:
    json.dump(items, f, indent=4)

In [59]:
# no correr, cambios a manita
delete = ['\\Bigg','\\left','\\right','\\Big','\\mathrm']
replace = {'\\to':'\\rightarrow', '\\gt':'>', '\\lt':'<'}
add = ['_','{','}','=','(',')','-','+','^','[',']', ',']
count = 0
for key in items:
    for dele in delete:
        if dele in items[key]:
            items[key] = items[key].replace(dele, "")
    for rep in replace:
        if (rep in items[key]):
            items[key] = items[key].replace(rep, replace[rep])
    for it in add:
        if (it in items[key]):
            items[key] = items[key].replace(it, " "+it+" ")
    if ('\\' in items[key]):
        items[key] = items[key].replace('\\', " \\")
    items[key] = re.split(r'\s+', items[key])
    count = 0
    for a in items[key]:
        if ('\\' not in a and len(a)>1):
            uno = items[key][:count]
            dos = re.split('', a)[1:-1]
            tres = items[key][count+1:]
            count+= len(dos)-1
            uno.extend(dos)
            uno.extend(tres)
            items[key] = uno
        count+=1
    if (items[key][-1]==""):
        items[key] = items[key][:-1]
    if (items[key][0]==""):
        items[key] = items[key][1:]
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\labels.json", 'w') as f:
    json.dump(items, f, indent=4)

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

num_words = 1000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

items = {}
# antes hay que correr la celda anterior
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\labels.json", 'r') as f:
    items = json.load(f)
tokens = {}
count = 0
for key in items:
    items[key] = ' '.join(map(str, items[key]))
    items[key] = '<start> '+items[key] + ' <end>'
    items[key] = re.sub(r"\s+", " ", items[key])
keys = list(items.keys())
data = [ items[key] for key in keys ]
print(data[0])
tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token, filters='', lower=False)
tokenizer.fit_on_texts(data)

word_index = tokenizer.word_index
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\tokens.json", 'w') as f:
    json.dump(word_index, f, indent=4)

train_sequences = tokenizer.texts_to_sequences(data)

maxlen = max([len(x) for x in train_sequences])

train_padded = pad_sequences(train_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)

<start> S = ( \sum _ { i = 1 } ^ { n } \theta _ i - ( n - 2 ) \pi ) r ^ 2 <end>


In [72]:
tokens = {}
files_imgs = {}
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\tokens.json", 'r') as f:
    tokens = json.load(f)
tokensPerFile = {}
items = {}
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\labels.json", 'r') as f:
    items = json.load(f)
for key in items:
    newKey = LABELS_PATH +'\\images'+ key[len(LABELS_PATH)+11:][:-6]+'.jpg'
    tokensPerFile[newKey] = []
    files_imgs[newKey] = key
    for command in items[key]:
        tokensPerFile[newKey].append(tokens[command])
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\labelsPerFile.json", 'w') as f:
    json.dump(tokensPerFile, f, indent=4)
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\files_img.json", 'w') as f:
    json.dump(files_imgs, f, indent=4)

In [12]:
#FCN + GRU (attention) 
# checar que onda con los filtros
# vector de cuatro en output
class FCN_encoder(tf.keras.Model):
    def __init__(self, dropout_rate = 0.2):
        super(FCN_encoder, self).__init__()
        
        # bloque de convolucion 1, 32 filtros
        self.conv_1_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1)
        self.batch_1_1 = tf.keras.layers.BatchNormalization()
        self.act_1_1 = tf.keras.layers.Activation('relu')
        self.conv_1_2 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1)
        self.batch_1_2 = tf.keras.layers.BatchNormalization()
        self.act_1_2 = tf.keras.layers.Activation('relu')
        self.conv_1_3 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1)
        self.batch_1_3 = tf.keras.layers.BatchNormalization()
        self.act_1_3 = tf.keras.layers.Activation('relu')
        self.conv_1_4 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1)
        self.drop_1 = tf.keras.layers.Dropout(dropout_rate)
        self.batch_1_4 = tf.keras.layers.BatchNormalization()
        self.act_1_4 = tf.keras.layers.Activation('relu')
        
        # maxpooling para reducir el tamaño
        self.maxPool_1 = tf.keras.layers.MaxPooling2D()
        
        # bloque convolucional 2, 64 filtros
        self.conv_2_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_2_1 = tf.keras.layers.BatchNormalization()
        self.act_2_1 = tf.keras.layers.Activation('relu')
        self.conv_2_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_2_2 = tf.keras.layers.BatchNormalization()
        self.act_2_2 = tf.keras.layers.Activation('relu')
        self.conv_2_3 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_2_3 = tf.keras.layers.BatchNormalization()
        self.act_2_3 = tf.keras.layers.Activation('relu')
        self.conv_2_4 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.drop_2 = tf.keras.layers.Dropout(dropout_rate)
        self.batch_2_4 = tf.keras.layers.BatchNormalization()
        self.act_2_4 = tf.keras.layers.Activation('relu')

        self.maxPool_2 = tf.keras.layers.MaxPooling2D()

        self.conv_3_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_3_1 = tf.keras.layers.BatchNormalization()
        self.act_3_1 = tf.keras.layers.Activation('relu')
        self.conv_3_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_3_2 = tf.keras.layers.BatchNormalization()
        self.act_3_2 = tf.keras.layers.Activation('relu')
        self.conv_3_3 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.batch_3_3 = tf.keras.layers.BatchNormalization()
        self.act_3_3 = tf.keras.layers.Activation('relu')
        self.conv_3_4 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)
        self.drop_3 = tf.keras.layers.Dropout(dropout_rate)
        self.batch_3_4 = tf.keras.layers.BatchNormalization()
        self.act_3_4 = tf.keras.layers.Activation('relu')

        self.maxPool_3 = tf.keras.layers.MaxPooling2D()

        self.conv_4_1 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2)
        self.batch_4_1 = tf.keras.layers.BatchNormalization()
        self.act_4_1 = tf.keras.layers.Activation('relu')
        self.conv_4_2 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2)
        self.batch_4_2 = tf.keras.layers.BatchNormalization()
        self.act_4_2 = tf.keras.layers.Activation('relu')
        self.conv_4_3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2)
        self.batch_4_3 = tf.keras.layers.BatchNormalization()
        self.act_4_3 = tf.keras.layers.Activation('relu')
        self.conv_4_4 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2)
        self.drop_4 = tf.keras.layers.Dropout(dropout_rate)
        self.batch_4_4 = tf.keras.layers.BatchNormalization()
        self.act_4_4 = tf.keras.layers.Activation('relu')

        self.maxPool_4 = tf.keras.layers.MaxPooling2D()
        
    def call(self, inputs):
        
        x = self.conv_1_1(inputs)
        x = self.batch_1_1(x)
        x = self.act_1_1(x)
        x = self.conv_1_2(x)
        x = self.batch_1_2(x)
        x = self.act_1_2(x)
        x = self.conv_1_3(x)
        x = self.batch_1_3(x)
        x = self.act_1_3(x)
        x = self.conv_1_4(x)
        x = self.drop_1(x)
        x = self.batch_1_4(x)
        x = self.act_1_4(x)
        x = self.maxPool_1(x)
        
        x = self.conv_2_1(x)
        x = self.batch_2_1(x)
        x = self.act_2_1(x)
        x = self.conv_2_2(x)
        x = self.batch_2_2(x)
        x = self.act_2_2(x)
        x = self.conv_2_3(x)
        x = self.batch_2_3(x)
        x = self.act_2_3(x)
        x = self.conv_2_4(x)
        x = self.drop_2(x)
        x = self.batch_2_4(x)
        x = self.act_2_4(x)
        x = self.maxPool_2(x)
        
        x = self.conv_3_1(x)
        x = self.batch_3_1(x)
        x = self.act_3_1(x)
        x = self.conv_3_2(x)
        x = self.batch_3_2(x)
        x = self.act_3_2(x)
        x = self.conv_3_3(x)
        x = self.batch_3_3(x)
        x = self.act_3_3(x)
        x = self.conv_3_4(x)
        x = self.drop_3(x)
        x = self.batch_3_4(x)
        x = self.act_3_4(x)
        x = self.maxPool_3(x)
        
        x = self.conv_4_1(x)
        x = self.batch_4_1(x)
        x = self.act_4_1(x)
        x = self.conv_4_2(x)
        x = self.batch_4_2(x)
        x = self.act_4_2(x)
        x = self.conv_4_3(x)
        x = self.drop_4(x)
        x = self.batch_4_3(x)
        x = self.act_4_3(x)
        x = self.conv_4_4(x)
        x = self.drop_4(x)
        x = self.batch_4_4(x)
        x = self.act_4_4(x)
        x = self.maxPool_4(x)
    
        return x
    
    def model(self):
        input = tf.keras.layers.Input(shape=(512, 1024, 1))
        return tf.keras.Model(inputs = input, outputs = self.call(input) )
print(FCN_encoder().model().summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512, 1024, 1 0                                            
__________________________________________________________________________________________________
conv2d_32 (Conv2D)              (None, 510, 1022, 32 320         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_32 (BatchNo (None, 510, 1022, 32 128         conv2d_32[0][0]                  
__________________________________________________________________________________________________
activation_32 (Activation)      (None, 510, 1022, 32 0           batch_normalization_32[0][0]     
____________________________________________________________________________________________

In [13]:
class Attender(tf.keras.Model):
    def __init__(self):
        super(Attender, self).__init__()
        self.W_1 = tf.keras.layers.Dense(128)
        self.U_a = tf.keras.layers.Dense(128)
        self.V_a = tf.keras.layers.Dense(1)
        self.F = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=2)
    
    def call(self, a, h):
        h_t = tf.expand_dims(h, 1)
        e_ti = self.V_a( (tf.nn.tanh( self.W_1(a) + self.U_a(h))))
        a_ti = tf.nn.softmax(e_ti, axis=1)
        context = a_ti * a
        context = tf.reduce_sum(context, axis =1)
        return context, a_ti       

In [24]:
class GRU_decoder(tf.keras.Model):
    def __init__(self, dimension, units, label_len):
        super(GRU_decoder, self).__init__()
        self.units = units
        self.embedding = tf.keras.layers.Embedding(label_len, dimension)
        self.gru = tf.keras.layers.GRU( self.units, return_sequences=True, return_state = True, recurrent_initializer='glorot_uniform')
        self.fc1 = tf.keras.layers.Dense(self.units)
        self.fc2 = tf.keras.layers.Dense(label_len)
        self.attention = Attender()
    def call(self, x, a, h):
        #print(a.shape, h.shape)
        context_v, attention_pro = self.attention(a,h)
        x = self.embedding(x)
        context_v = tf.expand_dims(context_v,1)
        print(context_v.shape, x.shape)
        x = tf.concat([context_v,x], axis =-1)
        out, state = self.gru(x)
        x = self.fc1(out)
        x = tf.reshape(x, (-1, x.shape[2]))
        x = self.fc2(x)
        return x, state, attention_pro
    
    def reset(self, batch):
        return tf.zeros((batch,1,3, self.units))

In [25]:
encoder = FCN_encoder()
decoder = GRU_decoder(256,128,len(list(word_index.keys())))

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')


def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [26]:
data = {}
LABELS_PATH = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML"
with open(LABELS_PATH+'\\labelsPerFile.json',  'r') as f:
    data = json.load(f)
    f.close()

In [27]:
#train padded
files_img = {}
with open(r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\files_img.json", 'r') as f:
    files_img = json.load(f)
    f.close()
keys = list(files_img.keys())
import pathlib
list_ds = tf.data.Dataset.list_files(list(data.keys()))
def getProcessedImages(f):
    index = keys.index(f)
    image = tf.io.read_file(f)
    image = tf.image.decode_jpeg(image, channels = 1)                       
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [512, 1024] ) 
    return image, train_padded[index] #[data[str(tf.constant(f).numpy())[2:-1].replace('\\\\', '\\') ]]
    
labeledDataset = list_ds.map(lambda x: tf.py_function(getProcessedImages, [x], [tf.float32, tf.int32]) ,num_parallel_calls=tf.data.AUTOTUNE)

for image, label in labeledDataset.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

Image shape:  (512, 1024, 1)
Label:  [ 4  2 64 11  2 19 17 31 31 67 59  9  2 52  3  3  2  2 55  3 11  2 19  3
  3  3  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [28]:
labeledDataset = labeledDataset.batch(16)
labeledDataset = labeledDataset.prefetch(buffer_size = tf.data.AUTOTUNE)

'''for image, label in labeledDataset.batch(64).take( 1 ):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())'''

'for image, label in labeledDataset.batch(64).take( 1 ):\n    print("Image shape: ", image.numpy().shape)\n    print("Label: ", label.numpy())'

In [29]:
import random
def train(image, groundTruth):
    loss = 0
    hidden = decoder.reset(groundTruth.shape[0])
    input_decoder = tf.expand_dims([word_index['<start>']] * groundTruth.shape[0], 1)
    print(input_decoder.shape)
    with tf.GradientTape() as tape:
        feature_map = encoder(image)
        for i in range(1, groundTruth.shape[1]):
            print(i)
            pred, hidden, _ = decoder(input_decoder, feature_map, hidden)
            loss += loss_function(target[:, i], predictions)
            dec_input = tf.expand_dims(target[:, i], 1)

    total_loss = (loss / int(target.shape[1]))

    trainable_variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, trainable_variables)

    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss, total_loss

In [30]:
def trainData(dataset, EPOCHS = 1):
    for epoch in range(EPOCHS):
        total_loss = 0
      
        for (batch, (img_tensor, target)) in enumerate(dataset):
            batch_loss, t_loss = train(img_tensor, target)
            total_loss += t_loss
          
            if batch % 100 == 0:
                average_batch_loss = batch_loss.numpy()/int(target.shape[1])
        print('Epoch {0:d}/{1:d}'.format(epoch+1, EPOCHS), ": {0:.3f}sec".format((end_time - start_time)))
        print('===============>  train-loss=%.3f' % (total_loss/num_steps))

In [31]:
trainData(labeledDataset)

(16, 1)


ResourceExhaustedError: OOM when allocating tensor with shape[16,248,504,64] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:Relu]

In [10]:
'''
def preTraining(dataSet, cache=True, shuffle_buffer_size=1000):
    dataSet = dataSet.shuffle( buffer_size=shuffle_buffer_size )
    #dataSet = dataSet.repeat()
    dataSet = dataSet.batch(16) #batch size =32
    #dataSet = dataSet.prefetch(buffer_size = tf.data.AUTOTUNE)
    return dataSet
trainDS = preTraining(labeledDataset)
#image_batch, label_batch = next(iter(trainDS))
#show_batch(image_batch.numpy(), label_batch.numpy())'''

'\ndef preTraining(dataSet, cache=True, shuffle_buffer_size=1000):\n    dataSet = dataSet.shuffle( buffer_size=shuffle_buffer_size )\n    #dataSet = dataSet.repeat()\n    dataSet = dataSet.batch(16) #batch size =32\n    #dataSet = dataSet.prefetch(buffer_size = tf.data.AUTOTUNE)\n    return dataSet\ntrainDS = preTraining(labeledDataset)\n#image_batch, label_batch = next(iter(trainDS))\n#show_batch(image_batch.numpy(), label_batch.numpy())'

In [14]:
# convert to grayscale
PATH = r"G:\\Documents\\CROHME\\CROHME\\CROHME2013_data\\TrainINKML\\images\\expressmatch\\101_alfonso.jpg"
image = cv.imread(PATH)
resized = cv.resize(image, (1024,512), interpolation = cv.INTER_AREA)
 
print('Resized Dimensions : ',resized.shape)
 
cv.imshow("Resized image", resized)
cv.waitKey(0)
cv.destroyAllWindows()
#grayImage = cv.cvtColor(image, cv.COLOR_BGR2GRAY)


Resized Dimensions :  (512, 1024, 3)


In [ ]:
path = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\images"
path2 = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\TrainINKML"
dirs = os.listdir(path)

for i in dirs:
    images = os.listdir(path+'\\'+i)
    files = os.listdir(path2+'\\'+i)
    images = set(map( lambda x: x[:-4], images ))
    files = set(map( lambda x: x[:-6], files ))
    print(images.difference(files))

In [ ]:
path = r"G:\Documents\CROHME\CROHME\CROHME2013_data\TrainINKML\images\expressmatch\65_alfonso.jpg"

In [6]:
a = tf.zeros((16,1,3, 128))
print(a.shape)

(16, 1, 3, 128)
